In [1]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

os_env = 'mac'

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/'

if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader_new_version0229 import *
import eicu_year_process

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/'

if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [2]:
if os_env == 'window':
    mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

else:
    mimic_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

In [3]:
mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]
dataset_mimic = dataset_mimic[~((dataset_mimic['INDEX']=='CASE3_CASE4_DF')&(dataset_mimic['Annotation']=='no_circ'))]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test = eicu_test[~((eicu_test['INDEX']=='CASE3_CASE4_DF')&(eicu_test['Annotation']=='no_circ'))]
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')
hosp_id, eicu_test = get_hospital_eicu.hospital(eicu_test)

eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test)
unitadmitsource, unittype, unitstaytype = get_hospital_eicu.make_eicu_dataset(eicu_type)

Trial:  0
Threshold 조정 + 0.05, 현재 한계값: 0.1
========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    174346
2.0     27608
3.0     70171
4.0     71699
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    17664
2.0     3030
3.0     9456
4.0     8278
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.51:0.08:0.20:0.21
테스트셋 클래스 비율: 0.46:0.08:0.25:0.22
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 5323
테스트셋 환자 수: 601
학습셋 체류 수: 5433
테스트셋 체류 수: 604
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.1
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.30
시도한 시행 횟수: 0


In [5]:
if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/incorperate_subtask_label.py'>

In [6]:
mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [8]:
los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [10]:
ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [11]:
ARDS8h_event_train = ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'
ARDS8h_event_valid = ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS8h_event_valid['ARDS_next_8h']='event'
ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'

In [12]:
sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')

In [13]:
SIC8h_event_train = sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'
SIC8h_event_valid = sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='SIC']
SIC8h_event_valid['SIC_next_8h']='event'
SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [14]:
reload(split)

<module 'split' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/split.py'>

In [15]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mort_mimic_train[~(mort_mimic_train['death']=='event')], mode = 'mimic')
X_valid_mort, y_valid_mort, valid_output_mort = split.split_X_Y_MORT(mort_mimic_valid[~(mort_mimic_valid['death']=='event')], mode = 'mimic')

X_train_los, y_train_los, _ = split.split_X_Y_LOS(los_mimic_train, mode = 'mimic')
X_valid_los, y_valid_los, valid_output_los = split.split_X_Y_LOS(los_mimic_valid, mode = 'mimic')

X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')
X_valid_ards8h, y_valid_ards8h, valid_output_ards8h = split.split_X_Y_ARDS8h(ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')

X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='no_SIC'], mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [17]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [18]:
mort_models, result_mort = Multiclass_evaluation.create_subtask(X_train_mort, y_train_mort, X_valid_mort, valid_output_mort, _,mode = 'mimic', type = 'binary', event_task = False)

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,F1 Score,Recall,Precision
0,xgb,0.650746,0.450126,0.448113,0.446359,0.449882
1,lgbm,0.681841,0.524490,0.465077,0.435395,0.499102
2,catboost,0.667413,0.475899,0.481180,0.485513,0.476923
3,rf,0.667164,0.442447,0.300209,0.224745,0.451969
4,dt,0.528856,0.321532,0.379423,0.453406,0.326197
5,svm-ovr,0.679104,0.394377,0.077253,0.042287,0.446281
6,lr,0.656219,0.390922,0.273396,0.203602,0.416000
7,naivebayes,0.317662,0.401766,0.482160,1.000000,0.317662
8,knn,0.589552,0.322881,0.327628,0.314800,0.341546


----------------------------------------------------------------------


In [129]:
result_mort.to_excel(excel_writer='mort_baseline_method.xlsx')

In [42]:
los_models, result_los = Multiclass_evaluation.create_subtask(X_train_los, y_train_los, X_valid_los, valid_output_los, _, mode = 'mimic', type = 'multi', event_task = False)

In [128]:
result_los.to_excel(excel_writer='los_baseline_method.xlsx')

In [174]:
ards8h_models, result_ards8h = Multiclass_evaluation.create_subtask(X_train_ards8h, y_train_ards8h, X_valid_ards8h, valid_output_ards8h, ARDS8h_event_valid, mode = 'mimic', type = 'binary', event_task='ARDS 8h')

ZeroDivisionError
|MIMIC-Validation|====================================================


,Model,AUPRC,ARDS score,Recall,Precision,Grecall
0,xgb,0.346432,0.263076,0.4355,0.419355,0.225694
1,lgbm,0.374790,0.241211,0.3387,0.475000,0.197917
2,catboost,0.368421,0.250000,0.3226,0.420690,0.211806
3,rf,0.437831,0.132075,0.2258,0.682927,0.097222
4,dt,0.340699,0.294487,0.4839,0.309701,0.288194
5,svm-ovr,0.371860,0.178521,0.2903,0.629032,0.135417
6,lr,0.412100,0.275424,0.4194,0.546218,0.225694
7,naivebayes,0.124763,0.000000,0.0000,0.000000,0.000000
8,knn,0.176442,0.133041,0.2581,0.169082,0.121528


----------------------------------------------------------------------


In [250]:
result_ards8h.to_excel(excel_writer='ards8h_baseline_method.xlsx')

In [251]:
X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train, mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [176]:
sic8h_models, result_sic8h = Multiclass_evaluation.create_subtask(X_train_sic8h, y_train_sic8h, X_valid_sic8h, valid_output_sic8h, SIC8h_event_valid, mode = 'mimic', type = 'binary', event_task='SIC 8h')

|MIMIC-Validation|====================================================


,Model,AUPRC,SIC score,Recall,Precision,Grecall
0,xgb,0.287631,0.165114,0.255814,0.319149,0.135952
1,lgbm,0.338607,0.139493,0.279070,0.382979,0.108761
2,catboost,0.392337,0.131797,0.186047,0.478261,0.099698
3,rf,0.316597,0.025974,0.046512,1.000000,0.018127
4,dt,0.338436,0.305717,0.511628,0.263285,0.329305
5,svm-ovr,0.283125,0.000000,0.000000,0.000000,0.000000
6,lr,0.267753,0.114913,0.162791,0.234848,0.093656
7,naivebayes,0.126672,0.318387,1.000000,0.125664,1.000000
8,knn,0.192156,0.126466,0.232558,0.263566,0.102719


----------------------------------------------------------------------


In [253]:
result_sic8h.to_excel(excel_writer='sic8h_baseline_method.xlsx')